In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import enum
import re
import nltk 
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to C:\Users\Atılay
[nltk_data]     Özgür\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Atılay
[nltk_data]     Özgür\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Atılay
[nltk_data]     Özgür\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [ ]:
#Veri setinin yüklenmesi ve örnek veri
dataset = pd.read_csv("data.csv",delimiter=";",header=None,names=["Review","Rating"])
print("Verisetinde {} adet cümle mevcut.".format(len(dataset)))
dataset.head()

In [ ]:
#Veri ön işleme
#Ön işleme öncesi örnek cümle
print(dataset['Review'].values[0],"\n\n")

WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('english')
print(stop_word_list)

def token(values):
    words = nltk.tokenize.word_tokenize(values)
    filtered_words = [word for word in words if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

#büyük harflerin küçük harfe çevrilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.lower())

# Özel karakterlerin(noktalama işareti vs) çıkartılması
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\W", " ", x))

# tek karakterlerin boşluk ile değiştirilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\s+[a-zA-Z]\s+", " ", x))

# en baştan tek kalan karakterlerin çıkartılması
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\^[a-zA-Z]\s+", " ", x))

# Birden fazla boşluğun tek boşlukla değiştirilmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"\s+", " ", x))

# b öneklerinin silinmesi
dataset['Review'] = dataset['Review'].apply(lambda x: re.sub(r"^b\s+", " ", x))

#fazladan boşlukların temizlenmesi
dataset['Review'] = dataset['Review'].apply(lambda x: x.strip())

#stopwordlerin temizlenmesi
dataset['Review'] = dataset['Review'].apply(lambda x: token(x))

data = dataset['Review'].values.tolist()
target = dataset['Rating'].values.tolist()

#Ön işleme sonrası aynı cümle
print("\n\n", data[0])

In [ ]:
#Cümlelerin eğitim ve test olarak ayrılması %90 Eğitim %10 test
ratio = int(len(data) * .90)
x_train, y_train = data[:ratio], target[:ratio]
y_train = np.array(y_train)
x_test, y_test   = data[ratio:], target[ratio:]
y_test = np.array(y_test)

print("{} adet cümle eğitim için kullanılacak.".format(len(x_train)))
print("{} adet cümle test için kullanılacak.".format(len(x_test)))

In [ ]:
#Cümlelerin içinde geçen kelimelerden 10000 kelimelik bir sözlük oluşturuluyor.
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data)
#tokenizer.word_index

#Cümleler sayılara dönüştürülüyor
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

#Cümlelerin önceki ve sonraki hallerinin görüntülenmesi
IDX = 0
print("Öncesi: {}".format(x_train[IDX]))
print("Sonrası: {}".format(np.array(x_train_tokens[IDX])))

In [ ]:
#RNN'e girdileri vermeden önce tamamının aynı boyutta olması gerekli. Bu sebeple aşağıdaki matematiksel işlemleri yapıyoruz.
total_sentences = x_train_tokens + x_test_tokens
num_tokens = np.array([len(tokens) for tokens in total_sentences])
#print(np.mean(num_tokens))
#print(np.std(num_tokens))
#print(np.max(num_tokens))
#print(np.min(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) # np.std = standart sapma
max_tokens = int(max_tokens)
print(max_tokens)
#Verinin ne kadarını bu kapsama aldığımızın ölçülmesi
print("%", round(np.sum(num_tokens < max_tokens) / len(num_tokens) * 100, 2))

In [ ]:
#Padding işlemi. Bulunan uzunluk değerine göre cümlelerin yeniden düzenlenmesi. Kısa olanların başına sıfır eklenmesi.
#Uzun olanlardan baştan silme yapılması
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad  = pad_sequences(x_test_tokens,  maxlen=max_tokens)

In [ ]:
#RNN oluşturma
#ardışık bir model
model = Sequential()
    
#her kelimeye karşılık gelen 50 uzunluğunda bir vektör oluşturulur. (Embedding matrisi)
embedding_size = 50
    
#matris kelime sayısı ve embedding büyüklüğünde olacak, yani 10bine 50 uzunluğunda 
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))
#LSTM layerlerinin eklenmesi
# 16 nöronlu LSTM (16 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(GRU(units=16, return_sequences=True))
# 8 nöronlu LSTM (8 outputlu , return_sequences=True demek output'un tamamını ver demek)
model.add(GRU(units=8, return_sequences=True))
# 4 nöronlu LSTM (4 outputlu , return_sequences=False yani default değer, tek bir output verecek)
model.add(GRU(units=4))
# Tek bir nörondan oluşan output layer'ı
model.add(Dense(1, activation='sigmoid'))

#modelin derlenmesi 
#iki sınıf olduğu için loss fonksiyonu olarak binary_crossentropy 
#modelin başarısını görmek için accuracy metrics
#optimizasyon algoritması
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
#Modelin özeti
model.summary()

In [ ]:
#model eğitimi, bir defa eğitimden geçmesi -> epoch , batch_size -> 256'şar 256'şar beslenecek.
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

In [ ]:
#Evaluate fonksiyonu yalnızca accuracy ve loss değerini döndürür
result = model.evaluate(x_test_pad, y_test)

num_true_sentence = int(len(x_test) * result[1])
print("Test verisindeki {} adet cümleden {} tanesi doğru bilindi.".format(len(x_test), num_true_sentence))

In [ ]:
#tek tek cümlelerin sonuçlarını görmek için predict metodu kullanılması
y_pred = model.predict(x_test_pad)

#Her cümle için çıktı 0 ile 1 arasındadır. 0 olumsuz 1 olumlu anlamındadır. 
#0.5 üzerini olumlu altını olumsuz olarak işaretleyelim.
y_pred = np.array([1 if p>0.5 else 0 for p in y_pred])

#Bir örnek üzerinde inceleyelim.
IDX = 0
sentence = x_test[IDX]
real_rate = y_test[IDX]
predicted_rate = y_pred[IDX]

print("Cümle: {} \nAsıl Etiket: {} \nÜretilen Etiket: {}".format(sentence, real_rate, predicted_rate))

In [ ]:
#Accuracy değeri hariç precision,recall ve f-measure değerlerine bakalım.
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))